### This notebook contains code for preprocessing data for the gpu only model.
The conda environment required for running this notebook can be installed and activated by running the following on the command line from within this folder: 
<code>conda env create -f evolution_env_gpu_only.yml </code>  \
<code>conda activate seq</code>


In [ ]:
import csv
import copy
with open('./*.shuffled.txt') as f: #replace with the path to your raw data
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)	

    

sequences = [di[0] for di in d]

for i in range(0,len(sequences)) : 
    if (len(sequences[i]) > 110) :
        sequences[i] = sequences[i][-110:]
    if (len(sequences[i]) < 110) : 
        #print i
        while (len(sequences[i]) < 110) :
            sequences[i] = 'N'+sequences[i]
            
            

A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}
worddim = len(mapper['A'])



def seq2feature(data,mapper,worddim):
    transformed = np.zeros([data.shape[0],1,len(data[0]),4] , dtype=np.bool )
    for i in range(data.shape[0]) :
        for j,k in enumerate(data[i]):
            transformed[i,0,j] = mapper[k] 

    return transformed
    


def get_rc(A):
    A_r = np.flip(A,2)
    A = A_r
    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    for i in range(A.shape[0]) : 
        for j in range(A.shape[2]) : 
            if np.array_equal(A[i][0][j] , A_onehot):
                A[i][0][j] = T_onehot
            elif np.array_equal(A[i][0][j] , C_onehot) :
                A[i][0][j] = G_onehot
            elif np.array_equal(A[i][0][j] , G_onehot) :
                A[i][0][j] = C_onehot
            elif np.array_equal(A[i][0][j] , T_onehot) :
                A[i][0][j] = A_onehot
            elif np.array_equal(A[i][0][j] , N_onehot) :
                A[i][0][j] = N_onehot

                
    return A



import numpy as np
seqdata = np.asarray(sequences)
seqdata_transformed = seq2feature(seqdata,mapper,worddim)
print seqdata_transformed.shape
with h5py.File('./onehot_sequences_bool.h5', 'w') as hf:
    hf.create_dataset("onehot_sequences_bool",  data=seqdata_transformed)
print type(seqdata_transformed[0][0][0][0])


expressions = [di[1] for di in d]
expdata = np.asarray(expressions)
expdata = expdata.astype('float')  
    
    
rc_onehot_sequences = get_rc(copy.deepcopy(onehot_sequences))


with h5py.File('./expression.h5', 'w') as hf:
    hf.create_dataset("expression",  data=expdata)  

with h5py.File('./rc_onehot_sequences_bool.h5', 'w') as hf:
    hf.create_dataset("rc_onehot_sequences_bool",  data=rc_onehot_sequences)      